# simplerReadPlotting.ipynb
## Marcus Viscardi,    July 17, 2023

I want to quickly take some tricks I learned from fourLibrary5TERA/ambiguousReads.py and apply them here.

The goal is to be able to plot reads from a locus based on their overlap of that locus and directionality. This will be a little messier, but important for all those genes that have a had time getting reads assigned to them! (*ets-4*)

Plan is to feed this a **target_gene** and a **bam file** then use pysam to rip all the relevant reads from the bam, and plot those!

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import pysam
import sys
from tqdm import tqdm
sys.path.insert(0, '/data16/marcus/scripts/nanoporePipelineScripts')
import nanoporePipelineCommon as npC
from finalizingReadAndCoveragePlotting_matplotlib import plot_reads, coverage_plotting_5tera

MIN_COVERAGE_OF_GENE = 50
print(f"Imports Complete @ {npC.get_dt(for_print=True)}")

In [ ]:
def create_cutdown_bam(target_genes: list, target_lib:str, save_dir='/tmp') -> (Path, pd.DataFrame):
    bam_file = Path(npC.pick_lib_return_path(npC.REV_CONVERSION_DICT[target_lib], output_dir_folder='cat_files', file_midfix='cat.sorted.mappedAndPrimary', file_suffix='.bam'))
    gtf_df = pd.read_parquet('/data16/marcus/genomes/'
                         'plus_cerENO2_elegansRelease100/'
                         '230327_allChrs_plus-cerENO2.gtf.parquet')
    short_gtf_df = gtf_df[(gtf_df['feature'] == 'gene') & (gtf_df['gene_name'].isin(target_genes))].copy()
    bam = pysam.AlignmentFile(bam_file, 'rb')
    cutdown_bam_path = f"{save_dir}/{npC.get_dt()}_{'_'.join(target_genes)}_{target_lib}.bam"
    cutdown_bam = pysam.AlignmentFile(cutdown_bam_path, 'wb',
                                      template=bam)
    for gene_id, gene_name in zip(short_gtf_df['gene_id'], short_gtf_df['gene_name']):
        print(f"Working on {gene_name} @ {npC.get_dt(for_print=True)}")
        chromosome, start, end, strand = \
        short_gtf_df.loc[short_gtf_df['gene_id'] == gene_id, ['chr', 'start', 'end', 'strand', ]].values[0]
        gene_is_reverse = (strand == '-')
        gene_is_forward = not gene_is_reverse
        read_iterator = tqdm(enumerate(bam.fetch(chromosome, start, end)))
        for i, read in read_iterator:
            if read.get_overlap(start, end) <= MIN_COVERAGE_OF_GENE:
                continue  # This will catch reads that are not in the gene! (50 is arbitrary)
                #           This is mainly because some reads "span" the gene, so they get fetched!
            if read.is_reverse != gene_is_reverse:
                continue  # This will catch reads that are in the wrong direction!
            read.set_tag('gA', gene_id, value_type='Z')
            read.set_tag('gN', gene_name, value_type='Z')
            read.set_tag('rS', read.reference_start, value_type='i')
            read.set_tag('rE', read.reference_end, value_type='i')
            read.set_tag('qL', read.query_length, value_type='i')
            cutdown_bam.write(read)
    for loop_bam in [cutdown_bam]:
        loop_bam.close()
        pysam.sort(loop_bam.filename, '-o', loop_bam.filename)
        pysam.index(loop_bam.filename)  # These samtools methods aren't in the pysam __init__ file, but they work!
    bam.close()
    print(f"Done. Saved to: {cutdown_bam_path}")
    return Path(cutdown_bam_path), short_gtf_df

In [ ]:
target_genes = (
    # 'ets-4',
    # 'zip-1',
    'xbp-1',
    'rps-22',
)
target_lib = 'newerS6'


create_cutdown_bam(target_genes, target_lib)

In [ ]:
def bam_tag_to_df(bam_object_or_path, tags=(), chr = None, start=None, stop = None):
    if not isinstance(bam_object_or_path, pysam.AlignmentFile):
        bam = pysam.AlignmentFile(bam_object_or_path, 'rb')
    else:
        bam = bam_object_or_path
    read_names = []
    chrs = []
    chr_starts = []
    strands = []
    cigars = []
    tags_dict = {tag: [] for tag in tags}
    for read in bam.fetch(chr, start, stop):
        read_names.append(read.query_name)
        chrs.append(read.reference_name)
        chr_starts.append(read.reference_start)
        strands.append(read.is_reverse)
        cigars.append(read.cigarstring)
        for add_tag in tags:
                tags_dict[add_tag].append(read.get_tag(add_tag))
    df = pd.DataFrame({'read_id': read_names,
                       'chr_id': chrs,
                       'chr_pos': chr_starts,
                       'strand': strands,
                       'cigar': cigars,
                       **tags_dict})
    df.strand.replace({True: '-', False: '+'}, inplace=True)
    return df

In [ ]:
quick_df = bam_tag_to_df(pysam.AlignmentFile(f"/tmp/{npC.get_dt()}_{'_'.join(target_genes)}_{target_lib}.bam", 'rb'), tags=['gA', 'gN', 't5', 'rS', 'rE']).rename(columns={'gA': 'gene_id', 'gN': 'gene_name'})
quick_df

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import re
get_dt = npC.get_dt


def _make_rectangle_patch(genome_start, length, y_center, thickness, color='gray'):
    return Rectangle((genome_start, y_center - (thickness / 2)), length, thickness,
                     facecolor=color,
                     edgecolor=color,
                     fill=True,
                     lw=0)


def _add_patches_from_cigars_and_gen_pos(axes, cigar, gen_start, y, color='black', plot_introns=True):
    # Parse the cigar string
    parsed_cigar = re.findall(rf'(\d+)([MDNSIX])', cigar)
    mdn_nums = [int(num) for num, char in parsed_cigar if char in "MDN"]
    gen_end = gen_start + sum(mdn_nums)
    genomic_read_length = gen_end - gen_start

    genome_loc = gen_start

    rectangle_patch_list = []
    first_n_length = 0
    for length, code in parsed_cigar:
        length = int(length)
        if code == 'S':
            pass
        elif code == 'M':
            rectangle_patch_list.append(_make_rectangle_patch(genome_loc, length, y, thickness=0.8))
            genome_loc += length
        elif code == 'D':
            if length < 50:
                rectangle_patch_list.append(_make_rectangle_patch(genome_loc, length, y, thickness=0.8))
            else:
                if plot_introns:
                    rectangle_patch_list.append(_make_rectangle_patch(genome_loc, length, y, thickness=0.001))
            genome_loc += length
        elif code == 'I':
            pass
        elif code == 'N':
            if plot_introns:
                rectangle_patch_list.append(_make_rectangle_patch(genome_loc, length, y, thickness=0.001))
            genome_loc += length
    axes.add_collection(PatchCollection(rectangle_patch_list, color=color))
    return genomic_read_length


def _add_patches_from_cigars_and_gen_pos_gtp(axes, cigar, gen_start, y, color='black', plot_introns=True):
    # Parse the cigar string
    parsed_cigar = re.findall(rf'(\d+)([MDNSIX])', cigar)
    mdn_nums = [int(num) for num, char in parsed_cigar if char in "MDN"]
    gen_end = gen_start + sum(mdn_nums)
    genomic_read_length = gen_end - gen_start

    genome_loc = gen_start

    rectangle_patch_list = []
    current_length = 0
    for length, code in parsed_cigar:
        length = int(length)
        if code == 'S':
            pass
        elif code == 'M':
            current_length += length
            genome_loc += length
        elif code == 'D':
            current_length += length
            genome_loc += length
        elif code == 'I':
            pass
        elif code == 'N':
            if plot_introns:
                # Only add the accumulated length as a single rectangle if we encountered matches or deletes
                if current_length > 0:
                    rectangle_patch_list.append(_make_rectangle_patch(genome_loc - current_length, current_length, y, thickness=0.8))
                    current_length = 0
                rectangle_patch_list.append(_make_rectangle_patch(genome_loc, length, y, thickness=0.001))
            genome_loc += length

    # Add the accumulated length as a single rectangle if we reached the end of the cigar
    if current_length > 0:
        rectangle_patch_list.append(_make_rectangle_patch(genome_loc - current_length, current_length, y, thickness=0.8))

    axes.add_collection(PatchCollection(rectangle_patch_list, color=color))

    return genomic_read_length

def subset_df(df, column, fractions):
    subset_df = df.groupby(column).apply(lambda x: x.sample(frac=fractions.get(x.name, 1))).reset_index(drop=True)
    return subset_df

def _row_apply_plot_cigar(row, axes, plot_introns=True, spacing_factor=1.0):
    index = row.name * spacing_factor
    cigar = row.cigar
    gen_start = row.ref_start
    is_adapted = row.t5

    if is_adapted == '-':
        color = 'black'
    else:
        color = 'red'
    return _add_patches_from_cigars_and_gen_pos_gtp(axes, cigar, gen_start, index,
                                                    color=color,
                                                    plot_introns=plot_introns)

def new_plot_reads_from_bam(target_lib_nickname,
                            target_gene_name,
                            subset_fraction={'+': 1, '-': 1},
                            spacing_factor=1.0,
                            save_dir=None,
                            ):
    # First lets make the tmp BAM file that only has the gene of interest:
    tmp_bam_path, short_gtf_df = create_cutdown_bam([target_gene_name], target_lib_nickname)
    bam_df = bam_tag_to_df(tmp_bam_path, tags=['gA', 'gN', 't5', 'rS', 'rE', 'qL'])
    bam_df.rename(columns={'gA': 'gene_name',
                           'gN': 'gene_id',
                           't5': 't5',
                           'rS': 'ref_start',
                           'rE': 'ref_end',
                           'qL': 'read_length'},
                  inplace=True)
    
    bam_df = subset_df(bam_df, "t5", subset_fraction)
    read_count = bam_df.shape[0]
    print(f"{read_count:>6,} reads in {target_lib_nickname} that match {target_gene_name}")
    
    # Get some gene info:
    gene_id, chromosome, genomic_start, genomic_end, gene_strand = short_gtf_df.loc[short_gtf_df['gene_name'] == target_gene_name, ['gene_id', 'chr', 'start', 'end', 'strand']].values[0]
    # Now lets plot the reads:
    fig, ax = plt.subplots(figsize=(5, (read_count * 0.08 * spacing_factor)))
    
    if gene_strand == "-":
        sort_order = ["t5", "ref_end", "ref_start", "read_length"]
        sort_order_ascending = [False, True, False, False]
    else:  # gene_strand == "+":
        sort_order = ["t5", "ref_start", "ref_end", "read_length"]
        sort_order_ascending = [False, False, False, False]
    
    bam_df = bam_df.sort_values(sort_order, ascending=sort_order_ascending).reset_index(drop=True)
    tqdm.pandas(desc="Plotting Reads...")
    bam_df.progress_apply(lambda row: _row_apply_plot_cigar(row, ax, spacing_factor=spacing_factor), axis=1)

    ax.set_xlim(genomic_start, genomic_end)
    
    number_of_plotted_reads = bam_df.shape[0]
    ax.set_ylim(-1, number_of_plotted_reads + 1)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.tight_layout()
    if save_dir is not None:
        save_path = f"{save_dir}/{get_dt()}_{target_lib_nickname}_{target_gene_name}.readPlotting"
        for suffix in ["png", "svg"]:
            plt.savefig(f"{save_path}.{suffix}", dpi=300)
        print(f"Saved images to: {save_path}.png/svg")
    plt.show()
    return bam_df

In [ ]:

sup_save_dir = f"/home/marcus/Insync/mviscard@ucsc.edu/Google Drive/insync_folder/NMD_cleavage_and_deadenylation_paper/raw_figures_from_python/{get_dt()}_readPlots"
for gene in [# "ets-4",
             # "xbp-1",
             # "zip-1",
             "rps-22",
             ]:
    save_dir = f"{sup_save_dir}/{gene}"
    Path(save_dir).mkdir(exist_ok=True, parents=True)
    for lib in ["newerN2",
                # "oldN2",
                "newerS6",
                # "oldS6",
                "newerS5",
                ]:
        new_plot_reads_from_bam(lib, gene,
                                subset_fraction={"+": 0.75, "-": 0.01},
                                save_dir=save_dir,
                                spacing_factor=1.2,)